In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

In [2]:
simus = ['ssp126',
         'ssp370',
         'ssp585',
         'hist-GHG',
         'hist-aer']
data_path = "/Users/chubb/DSC_180A/ClimateBench/data_preprocessed/train_val/"

In [3]:
X_train = []
Y_train = []

for i, simu in enumerate(simus):

    input_name = 'inputs_' + simu + '.nc'
    output_name = 'outputs_' + simu + '.nc'

    # Just load hist data in these cases 'hist-GHG' and 'hist-aer'
    if 'hist' in simu:
        # load inputs 
        input_xr = xr.open_dataset(data_path + input_name)
            
        # load outputs                                                             
        output_xr = xr.open_dataset(data_path + output_name).mean(dim='member')
        output_xr = output_xr.assign({"pr": output_xr.pr * 86400,
                                      "pr90": output_xr.pr90 * 86400}).rename({'lon':'longitude', 
                                                                               'lat': 'latitude'}).transpose('time','latitude', 'longitude').drop(['quantile'])
    
    # Concatenate with historical data in the case of scenario 'ssp126', 'ssp370' and 'ssp585'
    else:
        # load inputs 
        input_xr = xr.open_mfdataset([data_path + 'inputs_historical.nc', 
                                    data_path + input_name]).compute()
            
        # load outputs                                                             
        output_xr = xr.concat([xr.open_dataset(data_path + 'outputs_historical.nc').mean(dim='member'),
                               xr.open_dataset(data_path + output_name).mean(dim='member')],
                               dim='time').compute()
        output_xr = output_xr.assign({"pr": output_xr.pr * 86400,
                                      "pr90": output_xr.pr90 * 86400}).rename({'lon':'longitude', 
                                                                               'lat': 'latitude'}).transpose('time','latitude', 'longitude').drop(['quantile'])

    print(input_xr.dims, simu)
        # Append to list 
    X_train.append(input_xr)
    Y_train.append(output_xr)   

Frozen({'time': 251, 'longitude': 144, 'latitude': 96}) ssp126
Frozen({'time': 251, 'longitude': 144, 'latitude': 96}) ssp370
Frozen({'time': 251, 'longitude': 144, 'latitude': 96}) ssp585
Frozen({'time': 165, 'longitude': 144, 'latitude': 96}) hist-GHG
Frozen({'time': 165, 'longitude': 144, 'latitude': 96}) hist-aer


In [4]:
def normalize(data, var, meanstd_dict):
    mean = meanstd_dict[var][0]
    std = meanstd_dict[var][1]
    return (data - mean)/std

def unnormalize(data, var, meanstd_dict):
    mean = meanstd_dict[var][0]
    std = meanstd_dict[var][1]
    return data * std + mean
len_historical = 165

In [5]:
# Compute mean/std of each variable for the whole dataset
meanstd_inputs = {}

for var in ['CO2', 'CH4', 'SO2', 'BC']:
    # To not take the historical data into account several time we have to slice the scenario datasets
    # and only keep the historical data once (in the first ssp index 0 in the simus list)
    array = np.concatenate([X_train[i][var].data for i in [0, 3, 4]] + 
                           [X_train[i][var].sel(time=slice(len_historical, None)).data for i in range(1, 3)])
    print((array.mean(), array.std()))
    meanstd_inputs[var] = (array.mean(), array.std())

(1074.172303244536, 1755.690699230666)
(0.1927369743762821, 0.18457590641432994)
(2.5623359997066755e-12, 2.250114566783271e-11)
(1.4947905009818064e-13, 1.0313342554838387e-12)


In [6]:
X_train_norm = [] 
for i, train_xr in enumerate(X_train): 
    for var in ['CO2', 'CH4', 'SO2', 'BC']: 
        var_dims = train_xr[var].dims
        train_xr=train_xr.assign({var: (var_dims, normalize(train_xr[var].data, var, meanstd_inputs))}) 
    X_train_norm.append(train_xr)

In [7]:
slider = 10 # years moving temporal window 
# Functions for reshaping the data 
def input_for_training(X_train_xr, skip_historical=False, len_historical=None): 
    
    X_train_np =  X_train_xr.to_array().transpose('time', 'latitude', 'longitude', 'variable').data

    time_length = X_train_np.shape[0]
    # If we skip historical data, the first sequence created has as last element the first scenario data point
    if skip_historical:
        X_train_to_return = np.array([X_train_np[i:i+slider] for i in range(len_historical-slider+1, time_length-slider+1)])
    # Else we just go through the whole dataset historical + scenario (does not matter in the case of 'hist-GHG' and 'hist_aer')
    else:
        X_train_to_return = np.array([X_train_np[i:i+slider] for i in range(0, time_length-slider+1)])
    
    return X_train_to_return 


def output_for_training(Y_train_xr, var, skip_historical=False, len_historical=None): 
    Y_train_np = Y_train_xr[var].data
    
    time_length = Y_train_np.shape[0]
    
    # If we skip historical data, the first sequence created has as target element the first scenario data point
    if skip_historical:
        Y_train_to_return = np.array([[Y_train_np[i+slider-1]] for i in range(len_historical-slider+1, time_length-slider+1)])
    # Else we just go through the whole dataset historical + scenario (does not matter in the case of 'hist-GHG' and 'hist_aer')
    else:
        Y_train_to_return = np.array([[Y_train_np[i+slider-1]] for i in range(0, time_length-slider+1)])
    
    return Y_train_to_return

In [8]:
var_to_predict =  'tas'
# skip_historical set to (i < 2) because of the order of the scenario and historical runs in the X_train and Y_train lists.
# In details: ssp126 0, ssp370 1 = skip historical part of the data, ssp585 2, hist-GHG 3 and hist-aer 4 = keep the whole sequence
X_train_all = np.concatenate([input_for_training(X_train_norm[i], skip_historical=(i<2), len_historical=len_historical) for i in range(len(simus))], axis = 0)
Y_train_all = np.concatenate([output_for_training(Y_train[i], var_to_predict, skip_historical=(i<2), len_historical=len_historical) for i in range(len(simus))], axis=0)
print(X_train_all.shape)
print(Y_train_all.shape)

(726, 10, 96, 144, 4)
(726, 1, 96, 144)


In [8]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, Input, Reshape, AveragePooling2D, TimeDistributed, LSTM, GlobalAveragePooling2D
from tensorflow.keras.regularizers import l2

import random 
seed = 6
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

In [9]:
class PINN(tf.Module):
    def __init__(self, X_train, Y_train, slider=10):
        super().__init__()
        self.slider = slider
        self.input_train = X_train
        self.output_train = Y_train
        self.model = self.cnn_model()

    def cnn_model(self):
        cnn_model = Sequential()
        cnn_model.add(Input(shape=(self.slider, 96, 144, 4)))
        cnn_model.add(TimeDistributed(Conv2D(20, (3, 3), padding='same', activation='relu'), input_shape=(self.slider, 96, 144, 4)))
        cnn_model.add(TimeDistributed(AveragePooling2D(2)))
        cnn_model.add(TimeDistributed(GlobalAveragePooling2D()))
        cnn_model.add(LSTM(25, activation='relu'))
        cnn_model.add(Dense(1*96*144))
        cnn_model.add(Activation('linear'))
        cnn_model.add(Reshape((1, 96, 144)))
        return cnn_model

    def compile_model(self, optimizer='rmsprop', metrics=None):
        self.model.compile(optimizer=optimizer, loss=self.PINN_loss, metrics=metrics)

    def summary(self):
        self.model.summary()

    def PINN_loss(self, y_t, y_p):
        error = y_t - y_p
        # MSE between training and prediction
        mse = tf.reduce_mean(tf.square(error)) 
        # FaIR loss
        C0 = 598
        E = self.input_train[:, :, :, :, 0:1]
        C = C0 + E
        f1 = 4.57
        f2 = 0
        f3 = 0.086
        F = np.add(np.multiply(f1, np.log(np.divide(C,C0))), np.multiply(f3, np.subtract(np.sqrt(C), np.sqrt(C0))))
        qj = 0.3
        T = np.multiply(qj, F)
        dj = 10
        fair= tf.math.divide(T - tf.reduce_mean(y_p), dj)
        fair_mse = tf.reduce_mean(tf.square(fair))
        total = mse + fair_mse
        return total

    
    def fit(self, use_multiprocessing=True, batch_size=10, epochs=30, verbose=1):
        self.model.fit(self.input_train, self.output_train, use_multiprocessing=use_multiprocessing, batch_size=batch_size, epochs=epochs, verbose=verbose)


In [29]:
X_test = xr.open_mfdataset([data_path + 'inputs_historical.nc',
                            data_path + 'inputs_ssp245.nc']).compute()
model = PINN(X_train=X_train_all, Y_train=Y_train_all)
model.compile_model()

In [30]:
model.fit(epochs=15)

Epoch 1/15
73/73 [==============================] - 128s 565ms/step - loss: 1.6881
Epoch 2/15
73/73 [==============================] - 41s 565ms/step - loss: 0.5168
Epoch 3/15
73/73 [==============================] - 41s 563ms/step - loss: 0.4115
Epoch 4/15
73/73 [==============================] - 41s 568ms/step - loss: 0.4027
Epoch 5/15
73/73 [==============================] - 42s 569ms/step - loss: 0.3920
Epoch 6/15
73/73 [==============================] - 40s 553ms/step - loss: 0.3881
Epoch 7/15
73/73 [==============================] - 41s 562ms/step - loss: 0.3665
Epoch 8/15
73/73 [==============================] - 41s 558ms/step - loss: 0.3375
Epoch 9/15
73/73 [==============================] - 41s 560ms/step - loss: 0.3272
Epoch 10/15
73/73 [==============================] - 42s 580ms/step - loss: 0.3214
Epoch 11/15
73/73 [==============================] - 40s 542ms/step - loss: 0.3153
Epoch 12/15
73/73 [==============================] - 42s 572ms/step - loss: 0.3118
Epoch 13/15


In [ ]:
# Normalize data 
for var in ['CO2', 'CH4', 'SO2', 'BC']: 
    var_dims = X_test[var].dims
    X_test = X_test.assign({var: (var_dims, normalize(X_test[var].data, var, meanstd_inputs))}) 
    
X_test_np = input_for_training(X_test, skip_historical=False, len_historical=len_historical)

[[[[[-0.61171595 -0.1138758  -0.8746051  -0.14493754]
    [-0.61171595 -0.1138758  -0.8746051  -0.14493754]
    [-0.61171595 -0.1138758  -0.8746051  -0.14493754]
    ...
    [-0.61171595 -0.1138758  -0.8746051  -0.14493754]
    [-0.61171595 -0.1138758  -0.8746051  -0.14493754]
    [-0.61171595 -0.1138758  -0.8746051  -0.14493754]]

   [[-0.61171595 -0.1138758  -0.8746051  -0.14493754]
    [-0.61171595 -0.1138758  -0.8746051  -0.14493754]
    [-0.61171595 -0.1138758  -0.8746051  -0.14493754]
    ...
    [-0.61171595 -0.1138758  -0.8746051  -0.14493754]
    [-0.61171595 -0.1138758  -0.8746051  -0.14493754]
    [-0.61171595 -0.1138758  -0.8746051  -0.14493754]]

   [[-0.61171595 -0.1138758  -0.8746051  -0.14493754]
    [-0.61171595 -0.1138758  -0.8746051  -0.14493754]
    [-0.61171595 -0.1138758  -0.8746051  -0.14493754]
    ...
    [-0.61171595 -0.1138758  -0.8746051  -0.14493754]
    [-0.61171595 -0.1138758  -0.8746051  -0.14493754]
    [-0.61171595 -0.1138758  -0.8746051  -0.14493754]]

In [ ]:
m_pred = model.model.predict(X_test_np)
print(m_pred.shape)
m_pred = m_pred.reshape(m_pred.shape[0], m_pred.shape[2], m_pred.shape[3])
m_pred = xr.DataArray(m_pred, dims=['time', 'lat', 'lon'], coords=[X_test.time.data[slider-1:], X_test.latitude.data, X_test.longitude.data])
m_pred = m_pred.transpose('lat', 'lon', 'time').to_dataset(name=var_to_predict)
m_pred

8/8 [==============================] - 2s 174ms/step
(242, 1, 96, 144)


<xarray.Dataset>
Dimensions:  (time: 242, lat: 96, lon: 144)
Coordinates:
  * time     (time) int64 1859 1860 1861 1862 1863 ... 2096 2097 2098 2099 2100
  * lat      (lat) float64 -90.0 -88.11 -86.21 -84.32 ... 84.32 86.21 88.11 90.0
  * lon      (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
Data variables:
    tas      (lat, lon, time) float32 0.145 0.1449 0.1449 ... 6.702 6.729 6.754

In [10]:
vars_to_predict = ['tas', 'diurnal_temperature_range', 'pr', 'pr90']

# Open and reformat test data 
X_test = xr.open_mfdataset([data_path + 'inputs_historical.nc',
                            data_path + 'inputs_ssp245.nc']).compute()

# Normalize input data 
for var in ['CO2', 'CH4', 'SO2', 'BC']: 
    var_dims = X_test[var].dims
    X_test = X_test.assign({var: (var_dims, normalize(X_test[var].data, var, meanstd_inputs))}) 
    
X_test_np = input_for_training(X_test, skip_historical=False, len_historical=len_historical)

In [11]:
for var_to_predict in vars_to_predict:
    
    print(var_to_predict)
    
    # Data
    X_train_all = np.concatenate([input_for_training(X_train_norm[i], skip_historical=(i<2), len_historical=len_historical) for i in range(len(simus))], axis = 0)
    Y_train_all = np.concatenate([output_for_training(Y_train[i], var_to_predict, skip_historical=(i<2), len_historical=len_historical) for i in range(len(simus))], axis=0)
    print(X_train_all.shape)
    print(Y_train_all.shape)
    
    # Model    
    keras.backend.clear_session()
    model = PINN(X_train=X_train_all, Y_train=Y_train_all)
    model.compile_model()
    model.fit(epochs=30)
    
    # Make predictions using trained model 
    m_pred = model.model.predict(X_test_np)
    # Reshape to xarray 
    m_pred = m_pred.reshape(m_pred.shape[0], m_pred.shape[2], m_pred.shape[3])
    m_pred = xr.DataArray(m_pred, dims=['time', 'lat', 'lon'], coords=[X_test.time.data[slider-1:], X_test.latitude.data, X_test.longitude.data])
    xr_prediction = m_pred.transpose('lat', 'lon', 'time').sel(time=slice(2015,2101)).to_dataset(name=var_to_predict)

    if var_to_predict=="pr90" or var_to_predict=="pr":
        xr_prediction = xr_prediction.assign({var_to_predict: xr_prediction[var_to_predict] / 86400})

    # Save test predictions as .nc 
    if var_to_predict == 'diurnal_temperature_range':
        xr_prediction.to_netcdf("/Users/chubb/DSC_180A/ClimateBench/model_results/PINN/" + 'outputs_ssp245_predict_dtr.nc', 'w')
    else:
        xr_prediction.to_netcdf("/Users/chubb/DSC_180A/ClimateBench/model_results/PINN/" + 'outputs_ssp245_predict_{}.nc'.format(var_to_predict), 'w')
    xr_prediction.close()

tas
(726, 10, 96, 144, 4)
(726, 1, 96, 144)
Epoch 1/30
73/73 [==============================] - 89s 566ms/step - loss: 1.4898
Epoch 2/30
73/73 [==============================] - 41s 562ms/step - loss: 0.5073
Epoch 3/30
73/73 [==============================] - 42s 569ms/step - loss: 0.3967
Epoch 4/30
73/73 [==============================] - 41s 558ms/step - loss: 0.3971
Epoch 5/30
73/73 [==============================] - 41s 557ms/step - loss: 0.3857
Epoch 6/30
73/73 [==============================] - 42s 572ms/step - loss: 0.3663
Epoch 7/30
73/73 [==============================] - 40s 550ms/step - loss: 0.3331
Epoch 8/30
73/73 [==============================] - 40s 542ms/step - loss: 0.3180
Epoch 9/30
73/73 [==============================] - 39s 541ms/step - loss: 0.3167
Epoch 10/30
73/73 [==============================] - 40s 543ms/step - loss: 0.3146
Epoch 11/30
73/73 [==============================] - 40s 544ms/step - loss: 0.3110
Epoch 12/30
73/73 [==============================] -